In [1]:
import pandas as pd
import numpy as np


In [2]:
df_train = pd.read_csv('trainingDATA.csv', engine = 'python', on_bad_lines='skip')

In [3]:
df_train

,ID,title,summary,year_time,categories
0,2301.02657v1,TarViS: A Unified Approach for Target-based Vi...,The general domain of video segmentation is cu...,2023-01-06 18:59:52+00:00,cs.CV cs.AI cs.LG
1,2301.02642v1,Triple-stream Deep Metric Learning of Great Ap...,We propose the first metric learning system fo...,2023-01-06 18:36:04+00:00,cs.CV cs.AI cs.LG
2,2301.02610v1,Feedback-Gated Rectified Linear Units,Feedback connections play a prominent role in ...,2023-01-06 17:14:11+00:00,cs.NE cs.AI
3,2301.02593v1,Multi-Agent Reinforcement Learning for Fast-Ti...,To integrate high amounts of renewable energy ...,2023-01-06 16:41:51+00:00,cs.MA cs.AI cs.LG cs.SY
4,2301.02561v1,Multi-Vehicle Trajectory Prediction at Interse...,Traditional approaches to prediction of future...,2023-01-06 15:13:23+00:00,cs.RO cs.AI
...,...,...,...,...,...
8490,2205.15198v1,STN: Scalable Tensorizing Networks via Structu...,Deep neural networks (DNNs) have delivered a r...,2022-05-30 15:50:48+00:00,cs.CV cs.AI
8491,2205.15159v1,Updating belief functions over Belnap--Dunn logic,Belief and plausibility are weaker measures of...,2022-05-30 15:07:27+00:00,cs.AI cs.LO
8492,2205.15156v3,Towards Efficient 3D Object Detection with Kno...,Despite substantial progress in 3D object dete...,2022-10-14 02:06:11+00:00,cs.CV cs.AI cs.LG
8493,2206.14153v1,Multi-Fault Diagnosis Of Industrial Rotating M...,Industry 4.0 is an era of smart manufacturing....,2022-05-30 14:54:27+00:00,cs.AI


In [4]:
def remove_characters_from_text_doStemming_and_lower_it_(text):
    from nltk.stem.porter import PorterStemmer ##stemming purpose
    ps = PorterStemmer()
    review = re.sub('[^a-zA-Z!?]', ' ', text)
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)

    return review


def multilabel_encoder(y_real):
    from sklearn.preprocessing import MultiLabelBinarizer
    mlb = MultiLabelBinarizer()
    labels_encoded = mlb.fit_transform(y_real)
    return labels_encoded

In [5]:
df_tfidf_train = df_train.drop('ID', axis=1)
# df_tfidf_train[' title'] = df_tfidf_train[' title'].str.lower()
# df_tfidf_train['summary'] = df_tfidf_train['summary'].str.lower()

df_tfidf_train['content'] = df_tfidf_train[' title'] + ' ' + df_tfidf_train['summary']
df_tfidf_train['content'] = df_tfidf_train['content'].apply(lambda x: str(x))

import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')


df_tfidf_train['content'] = df_tfidf_train['content'].apply(remove_characters_from_text_doStemming_and_lower_it_)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#the above method is slow enough, we can do parallel computing also as....
import nltk
from nltk.tokenize import word_tokenize
import concurrent.futures


def remove_characters_from_text_doStemming_and_lower_it_(text):
    if isinstance(text, str):
        from nltk.stem.porter import PorterStemmer ##stemming purpose
        ps = PorterStemmer()
        review = re.sub('[^a-zA-Z!?]', ' ', text)
        review = review.lower()
        review = review.split()

        review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
        review = ' '.join(review)

        return review
    else:
        return text


def process_column(column, batch_size=100):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        for i in range(0, len(column), batch_size):
            batch = column[i:i + batch_size]
            yield list(executor.map(remove_characters_from_text_doStemming_and_lower_it_, batch))

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=650, use_idf=True)
X_tfidf = tfidf_vectorizer.fit_transform(df_tfidf_train['content'])

In [7]:
y_tfidf = df_tfidf_train['categories']

In [8]:
#doing a train test split
from sklearn.model_selection import train_test_split
X_tfidf_train, X_tfidf_test, y_tfidf_train, y_tfidf_test = train_test_split(X_tfidf, y_tfidf, test_size=0.25, random_state=42)

In [9]:
y_tfidf_train = y_tfidf_train.apply(lambda x: str(x))
y_tfidf_test = y_tfidf_test.apply(lambda x: str(x))
y_tfidf_train

,categories
3094,cs.CV cs.AI
110,cs.CV cs.AI
3213,cs.CL cs.AI
7150,cs.LG cs.AI cs.CC
1079,cs.DC cs.AI
...,...
5734,cs.LG cs.AI
5191,cs.CL cs.AI
5390,cs.AI cs.MA
860,cs.LG cs.AI cs.NE


In [10]:
y_tfidf_train = y_tfidf_train.apply(lambda x: x.split())
y_tfidf_test = y_tfidf_test.apply(lambda x: x.split())


In [11]:
y_tfidf_train.reset_index(drop=True, inplace=True)
y_tfidf_test.reset_index(drop=True, inplace=True)
y_tfidf_train

,categories
0,"[cs.CV, cs.AI]"
1,"[cs.CV, cs.AI]"
2,"[cs.CL, cs.AI]"
3,"[cs.LG, cs.AI, cs.CC]"
4,"[cs.DC, cs.AI]"
...,...
6366,"[cs.LG, cs.AI]"
6367,"[cs.CL, cs.AI]"
6368,"[cs.AI, cs.MA]"
6369,"[cs.LG, cs.AI, cs.NE]"


In [12]:
temp = []
for i in range(len(y_tfidf_train)):
    temp.append(y_tfidf_train[i])
y_tfidf_train = temp
y_tfidf_train

[['cs.CV', 'cs.AI'],
 ['cs.CV', 'cs.AI'],
 ['cs.CL', 'cs.AI'],
 ['cs.LG', 'cs.AI', 'cs.CC'],
 ['cs.DC', 'cs.AI'],
 ['cs.LG', 'cs.AI'],
 ['cs.CR', 'cs.AI', 'cs.GT'],
 ['cs.AI', 'cs.HC', 'cs.RO'],
 ['cs.AI', 'cs.GT', 'cs.MA'],
 ['cs.AI'],
 ['cs.SD', 'cs.AI', 'cs.LG', 'cs.MM'],
 ['cs.CY', 'cs.AI', 'cs.LG'],
 ['cs.SD', 'cs.AI', 'cs.CL', 'cs.LG', 'cs.NE'],
 ['cs.MA', 'cs.AI', 'cs.RO'],
 ['cs.MA', 'cs.AI', 'cs.GT', 'cs.LG'],
 ['cs.LG', 'cs.AI'],
 ['cs.LG', 'cs.AI'],
 ['cs.MA', 'cs.AI'],
 ['cs.CL', 'cs.AI', 'cs.LG'],
 ['cs.LG', 'cs.AI'],
 ['cs.AI', 'cs.LG'],
 ['cs.SI', 'cs.AI'],
 ['cs.CV', 'cs.AI'],
 ['cs.LG', 'cs.AI', 'cs.CV'],
 ['cs.CL', 'cs.AI'],
 ['cs.SD', 'cs.AI', 'cs.LG'],
 ['cs.CL', 'cs.AI', 'cs.LG'],
 ['cs.AI'],
 ['cs.AI', 'cs.RO'],
 ['cs.LG', 'cs.AI', 'cs.CL'],
 ['cs.LG', 'cs.AI'],
 ['cs.CL', 'cs.AI'],
 ['cs.SD', 'cs.AI', 'cs.LG'],
 ['cs.LG', 'cs.AI', 'cs.CL', 'cs.CV'],
 ['cs.CV', 'cs.AI'],
 ['cs.CL', 'cs.AI'],
 ['cs.HC', 'cs.AI'],
 ['cs.AI', 'cs.LG'],
 ['cs.CL', 'cs.AI'],
 ['cs.CV',

In [13]:
temp = []
for i in range(len(y_tfidf_test)):
    temp.append(y_tfidf_test[i])
y_tfidf_test = temp
y_tfidf_test

[['cs.CL', 'cs.AI'],
 ['cs.CL', 'cs.AI'],
 ['cs.CV', 'cs.AI'],
 ['cs.AI'],
 ['cs.AI', 'cs.CL', 'cs.HC', 'cs.LG'],
 ['cs.LG', 'cs.AI', 'cs.NE'],
 ['cs.SE', 'cs.AI', 'cs.LG', 'cs.PL'],
 ['cs.LG', 'cs.AI'],
 ['cs.CL', 'cs.AI'],
 ['cs.LG', 'cs.AI', 'cs.RO', 'cs.SY'],
 ['cs.CL', 'cs.AI'],
 ['cs.CV', 'cs.AI', 'cs.GR'],
 ['cs.LG', 'cs.AI', 'cs.RO'],
 ['cs.LG', 'cs.AI'],
 ['cs.LG', 'cs.AI'],
 ['cs.LG', 'cs.AI', 'cs.RO'],
 ['cs.HC', 'cs.AI'],
 ['cs.LG', 'cs.AI', 'cs.CV'],
 ['cs.AI', 'cs.LG'],
 ['cs.AI', 'cs.LG', 'cs.RO'],
 ['cs.RO', 'cs.AI', 'cs.CV', 'cs.LG'],
 ['cs.CL', 'cs.AI', 'cs.LG'],
 ['cs.CV', 'cs.AI'],
 ['cs.CV', 'cs.AI'],
 ['cs.AI', 'cs.NE'],
 ['cs.LG', 'cs.AI'],
 ['cs.LG', 'cs.AI'],
 ['cs.LG', 'cs.AI'],
 ['cs.CR', 'cs.AI', 'cs.LG'],
 ['cs.LG', 'cs.AI'],
 ['cs.HC', 'cs.AI'],
 ['cs.CL', 'cs.AI'],
 ['cs.MA', 'cs.AI', 'cs.CY', 'cs.GT', 'cs.LG'],
 ['cs.CL', 'cs.AI', 'cs.LG'],
 ['cs.CL', 'cs.AI'],
 ['cs.HC', 'cs.AI'],
 ['cs.LG', 'cs.AI', 'cs.CR'],
 ['cs.CL', 'cs.AI'],
 ['cs.RO', 'cs.AI', 'c

In [14]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
labels_encoded_train = mlb.fit_transform(y_tfidf_train)
labels_encoded_test = mlb.transform(y_tfidf_test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:900: UserWarning: unknown class(es) ['cs.CL,'] will be ignored
  warnings.warn(


In [15]:
array = mlb.classes_
array

array(['cs.AI', 'cs.AR', 'cs.CC', 'cs.CE', 'cs.CG', 'cs.CL', 'cs.CR',
       'cs.CV', 'cs.CY', 'cs.DB', 'cs.DC', 'cs.DL', 'cs.DM', 'cs.DS',
       'cs.ET', 'cs.FL', 'cs.GL', 'cs.GR', 'cs.GT', 'cs.HC', 'cs.IR',
       'cs.IT', 'cs.LG', 'cs.LO', 'cs.MA', 'cs.MM', 'cs.MS', 'cs.NA',
       'cs.NE', 'cs.NI', 'cs.OH', 'cs.OS', 'cs.PF', 'cs.PL', 'cs.RO',
       'cs.SC', 'cs.SD', 'cs.SE', 'cs.SI', 'cs.SY'], dtype=object)

In [1]:
from sklearn.linear_model import SGDClassifier
num_labels = labels_encoded_train.shape[1]
label_classifiers = {}

for label in range(num_labels):

    binary_target = labels_encoded_train[:, label]
    classifier = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=10, tol=None)

    classifier.fit(X_tfidf_train, binary_target)

    label_classifiers[label] = classifier

# Predict for each label separately
predictions = np.zeros((X_tfidf_test.shape[0], num_labels))
for label, classifier in label_classifiers.items():
    predictions[:, label] = classifier.predict(X_tfidf_test)


In [17]:
#Now using DL to predict the answer......

In [18]:
import tensorflow as tf

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import Adam

tf.__version__

'2.17.0'

In [20]:
#now creating a deep neural network to wok on tf-idf....
model_tfidf = Sequential()
model_tfidf.add(Dense(1000, input_shape = (X_tfidf_train.shape[1],),activation='relu'))
model_tfidf.add(Dropout(0.333))

model_tfidf.add(Dense(256, activation='relu'))
model_tfidf.add(Dropout(0.5))
model_tfidf.add(BatchNormalization())

model_tfidf.add(Dense(128, activation='relu'))
model_tfidf.add(Dropout(0.5))
model_tfidf.add(Dense(labels_encoded_train.shape[1], activation='sigmoid'))

model_tfidf.compile(optimizer=Adam(learning_rate=1e-3),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [21]:
history = model_tfidf.fit(X_tfidf_train, labels_encoded_train,
                    epochs=12,
                    batch_size=64,
                    validation_split=0.23)

Epoch 1/12
77/77 ━━━━━━━━━━━━━━━━━━━━ 7s 49ms/step - accuracy: 0.4310 - loss: 0.5366 - val_accuracy: 1.0000 - val_loss: 0.1410
Epoch 2/12
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - accuracy: 0.9950 - loss: 0.1146 - val_accuracy: 1.0000 - val_loss: 0.0951
Epoch 3/12
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - accuracy: 0.9990 - loss: 0.0929 - val_accuracy: 1.0000 - val_loss: 0.0880
Epoch 4/12
77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 47ms/step - accuracy: 0.9995 - loss: 0.0804 - val_accuracy: 1.0000 - val_loss: 0.0805
Epoch 5/12
77/77 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - accuracy: 0.9997 - loss: 0.0742 - val_accuracy: 1.0000 - val_loss: 0.0745
Epoch 6/12
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - accuracy: 0.9993 - loss: 0.0721 - val_accuracy: 1.0000 - val_loss: 0.0696
Epoch 7/12
77/77 ━━━━━━━━━━━━━━━━━━━━ 6s 42ms/step - accuracy: 0.9991 - loss: 0.0642 - val_accuracy: 1.0000 - val_loss: 0.0677
Epoch 8/12
77/77 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.9989 - loss: 0.0606 - val_accuracy: 1.0000 - v

In [23]:
y_pred = model_tfidf.predict(X_tfidf_test)

67/67 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step


In [24]:
y_pred

array([[9.9922013e-01, 2.9336493e-03, 2.4110521e-03, ..., 3.4540351e-02,
        2.6360901e-02, 1.5851878e-02],
       [9.9998230e-01, 3.8269485e-05, 5.2261341e-05, ..., 8.5983897e-04,
        1.0774538e-02, 1.3025026e-04],
       [9.9998993e-01, 7.7779638e-04, 1.7355834e-04, ..., 2.7257169e-04,
        1.3919257e-03, 4.7794016e-04],
       ...,
       [9.9996495e-01, 4.1519495e-04, 6.1767765e-05, ..., 3.0825415e-03,
        4.8749284e-03, 4.3772615e-04],
       [9.9998319e-01, 1.1369713e-03, 5.2759214e-04, ..., 7.6320861e-04,
        4.5052804e-02, 1.2608644e-03],
       [9.9990129e-01, 1.1275073e-04, 1.7599405e-04, ..., 5.5427966e-03,
        4.8141917e-03, 3.5447616e-03]], dtype=float32)